# futureEXPERT usage reference: Advanced stepwise invocation

## Login and expert client
futureEXPERT needs credentials for authentication. In order to log in, you create an authenticated client which handles all interactions with the future platform.

If you do not have an account for future yet, you can apply for a trial account [here](https://www.prognostica.de/de/future.html).

Additionally, if you are not subscribed for futureEXPERT, please contact us at support@future.prognostica.de

If you are using futureEXPERT for the first time, we recommend that you start with [Getting Started](simple_workflow.ipynb)

In [ ]:
from futureexpert import ExpertClient

# You can provide your credentials from an .env file (see .env.example)
import dotenv
dotenv.load_dotenv()

# Log in using your credentials or alternatively provide FUTURE_USER and FUTURE_PW via environment variables or .env file
client = ExpertClient(user='', password='')

## Run each step manually

Calling `client.create_forecast_from_raw_data` invokes a whole pipeline with several steps.

Instead, you can run each step individually in order to get intermediate results and fast feedback.

### Select your raw data

Two possible options:
- upload new data to future
- use already uploaded data

In [ ]:
import pprint
pp = pprint.PrettyPrinter(indent=4)  # Some basic formatting configuration for notebook/console output.

In [ ]:
# upload new data to future
upload_feedback = client.upload_data(source='../example-data/example_data.csv')
pp.pprint(upload_feedback)

In [ ]:
# receive all files you have already uploaded
input_data = client.client.get_user_inputs_for_group(client.group)
pp.pprint(input_data)

### Data preparation

The data preparation is divided into two steps:
- data defintion: checks if your date fits the given definiton 
- time series creation: creates the time series that will be used for the forecasting

In [ ]:
from futureexpert import *

data_definition = DataDefinition(remove_rows=[],
                                 remove_columns=[],
                                 date_columns=DateColumn(name='Date', format='%d.%m.%Y'),
                                 value_columns=[ValueColumn(name='Umsatz')],
                                 group_columns=[GroupColumn(name='land')])

user_input_id = upload_feedback['uuid']
file_id = upload_feedback['files'][0]['uuid']

data_definition_feedback = client.check_data_definition(user_input_id=user_input_id,
                                                        file_uuid=file_id,
                                                        data_definition=data_definition,
                                                        file_specification=FileSpecification(delimiter=";"))
pp.pprint(data_definition_feedback)

In [ ]:
config_ts_creation = TsCreationConfig(timeGranularity='monthly', valueColumnsToSave=[
                                      'Umsatz'], missingValueHandler='setToZero')

create_ts_feedback = client.create_time_series(config_ts_creation)
pp.pprint(create_ts_feedback)

### Start the forecast

The time series identification number received from the time series creation task are needed as input as well as the report configuration.

In [ ]:
fc_report_config = ReportConfig(title='My first report', forecasting=ForecastingConfig(fc_horizon=5))

version = create_ts_feedback['result']['tsVersion']['_id']
forecast_identifier = client.start_forecast(version=version, config=fc_report_config)